In [ ]:
from langgraph.graph import StateGraph, END  # Updated imports
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI


In [3]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [4]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")  # Using 3.5 for cost efficiency

In [ ]:
# Define our state type
from typing import TypedDict, Annotated, Literal
from typing_extensions import TypedDict

class AgentState(TypedDict):
    input: str
    classification: str
    response: str
    error: str
    logs: list[str]

In [9]:
from langgraph.graph import StateGraph, END  # Updated imports
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# Initialize the LLM
llm = ChatOpenAI(model="gpt-3.5-turbo")  # Using 3.5 for cost efficiency

# Define our state type
from typing import TypedDict, Annotated
from typing_extensions import TypedDict

class AgentState(TypedDict):
    input: str
    classification: str
    response: str

# Add validation logic

def validate_classification(state: AgentState):
    classification = state["classification"]
    valid = classification in ("freshwater", "saltwater")
    
    if not valid:
        return{"error": f"Invalid classification: {classification}"}
    return {"error": ""} # No error"]

# Define our nodes
def input_node(state: AgentState):
    return {"input": state["input"]}

def classification_node(state: AgentState):
    description = state["input"]
    response = llm.invoke([
        HumanMessage(content=f"""Classify this fish based on description. 
                     Only answer 'freshwater' or 'saltwater': {description}""")
    ])
    return {"classification": response.content.lower().strip()}

def response_node(state: AgentState):
    classification = state["classification"]
    description = state["input"]
    
    if "freshwater" in classification:
        prompt = f"Describe this freshwater fish in 1 sentence and give 3 key facts: {description}"
    else:
        prompt = f"Describe this saltwater fish in 1 sentence and give 3 key facts: {description}"
    
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"response": response.content}

def error_node(state: AgentState):
    response = llm.invoke([
        HumanMessage(content=f"""
            Correct this fish classification (answer ONLY 'freshwater' or 'saltwater'):
            Description: {state['input']}
            Invalid classification: {state['classification']}
            Error: {state['error']}
        """)
    ])
    return {"classification": response.content.lower().strip()}

# Build the graph
workflow = StateGraph(AgentState)  # Using StateGraph instead of Graph

# Add nodes
workflow.add_node("input", input_node)
workflow.add_node("classify", classification_node)
workflow.add_node("validate, validate_classification)  # Validation node")
workflow.add_node("error", error_node)
workflow.add_node("respond", response_node)

# Set up edges
workflow.set_entry_point("input")
workflow.add_edge("input", "classify")
workflow.add_edge("classify", "validate")  # Classify → Validate

# Conditional edge: Valid or not?
workflow.add_conditional_edges(
    "classify",
    lambda x: "respond",  # Our simple condition
    )
workflow.add_edge("error", "classify")  # After fixing, re-classify
workflow.add_edge("respond", END)

# Compile
app = workflow.compile()

# Run it!
result = app.invoke({"input": "A colorful fish with long flowing fins found in home aquariums"})
print(result["response"])

The Betta fish, also known as Siamese fighting fish, is a colorful freshwater fish with long flowing fins that is commonly found in home aquariums.
1. Betta fish are territorial and should be kept alone in a tank unless in a very large, heavily planted community tank.
2. They have a labyrinth organ that allows them to breathe air from the water's surface, making them able to survive in oxygen-poor environments.
3. Male Betta fish are known for their aggression towards other males, often engaging in intense fights to establish dominance.
